<a href="https://colab.research.google.com/github/benihime91/pytorch_retinanet/blob/master/nb/03_pascal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# What GPU do we have ?
! nvidia-smi

### **Standard Imports & Setup:**

**Setup:**


The cell below ensures that Google Collab doesn't disconnect to inactivity.

In [ ]:
# Ensure colab doesn't disconnect
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [ ]:
# install dependencies
! pip install pytorch-lightning wandb  --quiet
! pip install git+https://github.com/albumentations-team/albumentations --quiet

**Mount `GoogleDrive` & extract the Data:**  


The data is stored in the following paths:
- train_data : `/content/drive/My Drive/Pascal 2007 Data/pascal_voc_2007_train_val.zip`.
- test_data : `/content/drive/My Drive/Pascal 2007 Data/pascal_voc_2007_test_val.zip`.

These folders should contain images in a folder `/Images` and annotations in a folder `/Annotations`

In [ ]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Grab the Data
! unzip -qq /content/drive/My\ Drive/Pascal\ 2007\ Data/pascal_voc_2007_test.zip
! unzip -qq /content/drive/My\ Drive/Pascal\ 2007\ Data/pascal_voc_2007_train_val.zip

In [ ]:
# Clone the RetinaNet Repo:
! git clone https://github.com/benihime91/pytorch_retinanet.git

In [ ]:
# use wandb to track experiments : Comment this if not using wandb logger
! wandb login 'a74f67fd5fae293e301ea8b6710ee0241f595a63'

**Imports:**

In [ ]:
import warnings
import os
import sys
from typing import *
import time
import argparse

warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pickle


import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

# PyTorch Imports
import torch
from torch import nn
from torch.optim import *
from torch.utils.data import Dataset, DataLoader

# PyTorchLightning imports
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import (EarlyStopping, ModelCheckpoint, LearningRateLogger,)

# Import some usefull utilities from the RetinaNet Repo:
from pytorch_retinanet import DetectionDataset, Visualizer, Retinanet
from pytorch_retinanet.src.utils.coco_utils import CocoEvaluator, get_coco_api_from_dataset
from pytorch_retinanet.src.utils.general_utils import collate_fn


pl.seed_everything(123) # change this seed number to get different results
pd.set_option("display.max_colwidth", None)

### **Sanity Check:**

**Load in the Preprocessed Data:**

Run this [notebook](https://github.com/benihime91/pytorch_retinanet/blob/master/nbs/01_preprocess_pascal.ipynb) to process the data.

In [ ]:
# Path to the converted DataFrames

# NOTE: Update these path names from to the paths where your train , validation, test_data and 
# image_targets (in pickle format) is saved !
trn_csv_dir = '/content/drive/My Drive/Pascal 2007 Data/trn_data.csv' # path to the train csv file
val_csv_dir = '/content/drive/My Drive/Pascal 2007 Data/val_data.csv' # path to the valid csv file
tst_csv_dir = '/content/drive/My Drive/Pascal 2007 Data/tst_data.csv' # path to the test csv file
label_dir = "/content/drive/My Drive/Pascal 2007 Data/names.pkl" # path to label dictionary saved in pickle format

**Let's cross check the data to make sure everything is all right.. At the same time let's visualize some examples from the data ...**

In [ ]:
# Load in the DataFrames
trn_df = pd.read_csv(trn_csv_dir)
val_df = pd.read_csv(val_csv_dir)
tst_df = pd.read_csv(tst_csv_dir)

# Load in the Label Dict
label_dict = pickle.load(open(label_dir, "rb"))

In [ ]:
# Print out some information about the Data:
print('Num examples in train dataset :', len(trn_df.filename.unique()))
print('Num examples in valid dataset :', len(val_df.filename.unique()))
print('Num examples in test dataset  :', len(tst_df.filename.unique()))

**Train data:**

In [ ]:
trn_df.head() # train dataframe

**Validation Data:**

In [ ]:
val_df.head() # validation dataframe

**Test Data:**

In [ ]:
tst_df.head() # test dataframe

**Label Dictionary (Dicitionary containing the Labels)**

In [ ]:
label_dict # a dictionary which stores the mapping of target_labels to class_labels

**Instantiate `Visualizer` to display images with `bboxes`**

In [ ]:
# Instantiate the visualizer
viz = Visualizer(class_names=label_dict)

def display_random_image(df: pd.DataFrame) -> None:
    """
    Fn to display a random Image with bounding boxes drawn above it 
    from given pandas.Dataframe
    """
    n = np.random.randint(0, len(df))
    fname = df["filename"][n]
    boxes = df.loc[df["filename"] == fname][["xmin", "ymin", "xmax", "ymax"]].values
    labels = df.loc[df["filename"] == fname]["labels"].values
    
    viz.draw_bboxes(fname, boxes=boxes, classes=labels, figsize=(10, 10))

**View Random Images from the Dataset :**

In [ ]:
# Display random Image from the train set
display_random_image(trn_df)

In [ ]:
# Display random Image from the validation set
display_random_image(val_df)

In [ ]:
# Display random Image from the Test Dataset
display_random_image(tst_df)

### **Image Transformations:**


**Instantiate `transforms`:**

In [ ]:
def get_tfms() -> Dict[str, A.Compose]:
    """
    Returns a dictionary contatining albumentations 
    transformations for train,valid,test datasets.
    """
    # train transformations : [Modify this to add Transformations to train dataset] 
    trn_tfms = [
        A.HorizontalFlip(p=0.5),
        A.ToFloat(max_value=255.0, always_apply=True), # range the pixel values between [0, 1]
        ToTensorV2(always_apply=True),
    ]

    # validation transformations : [Transformations to the validation dataset]
    val_tfms = [
        A.ToFloat(max_value=255.0, always_apply=True),
        ToTensorV2(always_apply=True),
    ]

    # test transformations : [Transformations to the test dataset]
    tst_tfms = [
        A.ToFloat(max_value=255.0, always_apply=True),
        ToTensorV2(always_apply=True),                
    ]

    # transforms dictionary :
    transforms = {
        "train": A.Compose(trn_tfms, bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"]),),
        "valid": A.Compose(val_tfms, bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"]),),
        "test" : A.Compose(tst_tfms, bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"]),),
    }
    
    return transforms

### **Lightning Class:**

In this `notebook` we will use `PyTorchLightning` to train, validate and evaluate our model.

**Create `pl.LightningModule` instance :** 


Here we will instantiate a `pl.LightningModule` instance.The `LightningModule ` holds all the core research ingredients :
- The model
- The optimizers
- The train/ val/ test steps

**Lightning class:**  

To use pytorch-lightning, we need to define a main class, which has the following parts:

- `hparams`- This is optional parameter, but it better to use it - it is a dictionary with hyperparameters;
- `forward method` - making predictions with the model. The model itself can be defined outside this class;
- `prepare data` - preparing datasets;
- `train_dataloader`, `val_dataloader`, `test_dataloader` - these methods should return the relevant dataloaders;
- `configure_optimizers` - should return lists of optimizers and schedulers;
- `training_step` - define what happend inside the train loop;
- `validation_step` - define what happend inside the validation loop;
- `validation_epoch_end` - define what happend at epoch end;
- `test_step` - define what happend inside the test loop;
- `test_epoch_end` - define what happend at epoch end;

**In our specific case the `hparams` `Union[Dict, argparse.Namespace]` should contain the following fields:**

- `optimizer` `(torch.optim.Optimizer)` - Optimizer for the model
- `scheduler` `(Union[torch.optim.lr_scheduler, None])` - Scheduler for the `optimizer`, if no `scheduler` is used set `scheduler` to None.
- `trn_df` `(pd.DataFrame)` - train dataframe
- `trn_bs` `(int)`-train batch_size
- `val_df` `(pd.DataFrame)` - validation dataframe
- `val_bs` `(int)`-validation batch_size
- `test_df` `(pd.DataFrame)` - test dataframe
- `test_bs` `(int)`- test batch_size
- `iou_types` `(List)` - this parameter is used for evaluation using `COCO API` set it to `["bbox"]`.

In [ ]:
# Create pl.LightningModule instance
class DetectionModel(pl.LightningModule):
    def __init__(
        self, model: nn.Module, hparams: Union[Dict, argparse.Namespace]
    ) -> None:
        super(DetectionModel, self).__init__()
        self.model = model
        self.hparams = hparams

    def num_batches(self) -> List:
        """
        Returns a list containing the number of batches in train, 
        val & test dataloaders.
        """
        return [
            len(self.train_dataloader()),
            len(self.val_dataloader()),
            len(self.test_dataloader()),
        ]

    ##################################################################
    ############## Configure Optimizer & Schedulers ##################
    ##################################################################
    def configure_optimizers(self, *args, **kwargs):
        "instatiates optimizer & scheduler(s)"
        # optimizer
        optimizer = self.hparams.optimizer
        # scheduler
        scheduler = self.hparams.scheduler

        if scheduler is not None:
            return [optimizer], [scheduler]
        else:
            return [optimizer]

    ##################################################################
    ########################## preprare data #########################
    ##################################################################
    def prepare_data(self, stage=None):
        """
        load in the transformation & reads in the data from given paths.
        """
        # Instantiate Transforms:
        self.tfms = get_tfms()
        # Load in the DataFrames
        self.trn_df = pd.read_csv(self.hparams.trn_df)  # train dataframe
        self.val_df = pd.read_csv(self.hparams.val_df)  # valid dataframe
        self.test_df = pd.read_csv(self.hparams.test_df)  # test dataframe

    ##################################################################
    ############# Forward Pass of the Model ##########################
    ##################################################################
    def forward(self, xb, *args, **kwargs):
        "forward step"
        return self.model(xb)

    ##################################################################
    ########################### Trainining ###########################
    ##################################################################
    def train_dataloader(self, *args, **kwargs):
        "instantiate train dataloader"
        # instantiate the trian dataset
        train_ds = DetectionDataset(self.trn_df, self.tfms["train"])
        # load in the dataloader
        trn_dl = DataLoader(
            train_ds,
            batch_size=self.hparams.trn_bs,
            shuffle=True,
            collate_fn=collate_fn,
            pin_memory=True,
        )

        return trn_dl

    def training_step(self, batch, batch_idx, *args, **kwargs):
        "one training step"
        images, targets, _ = batch  # unpack the one batch from the DataLoader
        targets = [{k: v for k, v in t.items()} for t in targets]  # Unpack the Targets
        loss_dict = self.model(
            images, targets
        )  # Calculate Losses {regression_loss , classification_loss}
        losses = sum(loss for loss in loss_dict.values())  # Calculate Total Loss
        return {"loss": losses, "log": loss_dict, "progress_bar": loss_dict}

    ##################################################################
    ###################### Validation ################################
    ##################################################################
    def val_dataloader(self, *args, **kwargs):
        "instatiate validation dataloader"
        # instantiate the validaiton dataset
        val_ds = DetectionDataset(self.val_df, self.tfms["valid"])
        # instantiate dataloader
        loader = DataLoader(
            val_ds,
            batch_size=self.hparams.val_bs,
            shuffle=False,
            collate_fn=collate_fn,
        )
        
        # instantiate coco_api to track metrics
        coco = get_coco_api_from_dataset(
            loader.dataset
        )  # Convert dataset to COCO dataset format: for evaluation
        
        self.coco_evaluator = CocoEvaluator(
            coco, self.hparams.iou_types
        )  # Instantiate COCO Evaluator
        return loader

    def validation_step(self, batch, batch_idx, *args, **kwargs):
        "one validation step"
        images, targets, _ = batch
        targets = [{k: v for k, v in t.items()} for t in targets]
        outputs = self.model(images, targets)
        res = {
            target["image_id"].item(): output
            for target, output in zip(targets, outputs)
        }
        self.coco_evaluator.update(res)
        return {}

    def validation_epoch_end(self, outputs, *args, **kwargs):
        self.coco_evaluator.accumulate()
        self.coco_evaluator.summarize()
        metric = self.coco_evaluator.coco_eval["bbox"].stats[0]
        metric = torch.as_tensor(metric)
        logs = {"valid_mAP": metric}
        return {
            "valid_mAP": metric,
            "log": logs,
            "progress_bar": logs,
        }

    ##################################################################
    ######################## Test ####################################
    ##################################################################
    def test_dataloader(self, *args, **kwargs):
        "instatiate validation dataloader"
        # instantiate train dataset
        test_ds = DetectionDataset(self.test_df, self.tfms["test"])
        # instantiate dataloader
        loader = DataLoader(
            test_ds,
            batch_size=self.hparams.test_bs,
            shuffle=False,
            collate_fn=collate_fn,
        )
        # instantiate coco_api to track metrics
        coco = get_coco_api_from_dataset(loader.dataset)
        self.test_evaluator = CocoEvaluator(coco, self.hparams.iou_types)
        return loader

    def test_step(self, batch, batch_idx, *args, **kwargs):
        "one test step"
        images, targets, _ = batch
        targets = [{k: v for k, v in t.items()} for t in targets]
        outputs = self.model(images, targets)
        res = {
            target["image_id"].item(): output
            for target, output in zip(targets, outputs)
        }
        self.test_evaluator.update(res)
        return {}

    def test_epoch_end(self, outputs, *args, **kwargs):
        self.test_evaluator.accumulate()
        self.test_evaluator.summarize()
        metric = self.test_evaluator.coco_eval["bbox"].stats[0]
        metric = torch.as_tensor(metric)
        logs = {"test_mAP": metric}
        
        return {
            "test_mAP": metric,
            "log": logs,
            "progress_bar": logs,
        }

**To train a `LightningModule` we need to use the special `Trainer` class from `PyTorch Lightning`.**

### **Configurations :**

**Configure `LightningModule` and `Trainer` Configurations:** 

In this part we will create set up our trainining configuration for both the `LightningModule` & the `Trainer`.


**Configuration for `LightningModule`:**

In [ ]:
# Configurations for `LightningModule`

# ===================================================================================== #
# Number of Epochs to Train for :
# ===================================================================================== #
EPOCHS = 55 # set number of epochs to Train for

# ==================================================================================== #
# Instantiate model
# ==================================================================================== #
NUM_CLASSES = 20 # Total number of unique Targets
BACKBONE = 'resnet50' # backbone for RetinaNet Model
# load in the RetinaNet model
model = Retinanet(num_classes=NUM_CLASSES, backbone_kind=BACKBONE,)

# ==================================================================================== #
# Parameters for the Train, Validation & the Test Data
# ==================================================================================== #
# Train dataset Parametrs:
trn_df = trn_csv_dir # path to train_csv file
trn_bs = 4 # train batch_size

# Valid dataset parametrs:
val_df = val_csv_dir # path to validation_csv file
val_bs = 32 # validation batch_size

# Test dataset parametrs:
test_df = tst_csv_dir # path to test_csv file 
test_bs = 32 # test batch_size

# ==================================================================================== #
# Optimizer & Scheduler Patameters
# ==================================================================================== #
LR = 1e-03 # Optimizer learning_rate
WEIGHT_DECAY = 0.01  # optimizer weight_decay
MOMENTUM = 0.9 # optimzier momentum
params = [p for p in model.parameters() if p.requires_grad] # model parameters to train

# Instantiate Optimizer
optimizer = SGD(params, LR, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM) # Optimizer

# Instantiate scheduler
# Note: If no scheduler is used set `scheduler` to None
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[32,45], gamma=0.1,) # Scheduler
# convert scheduler to lightning format
if scheduler is not None:
    INTERVAL = "epoch" # scheduler interval wether after each 'step' for each 'epoch'
    scheduler = {"scheduler": scheduler, "interval": INTERVAL , "frequency": 1,}

# ==================================================================================== #
# set iou types:
# ==================================================================================== #
iou_types = ['bbox'] # Required for COCO eval

# ===================================================================================== #
# Create Dictionary to Store the arguments:
# ===================================================================================== #
hparams_dict = {
    'optimizer'     : optimizer,
    'scheduler'     : scheduler,
    'trn_df'        : trn_df,
    'trn_bs'        : trn_bs,
    'val_df'        : val_df,
    'val_bs'        : val_bs,
    'test_df'       : test_df,
    'test_bs'       : test_bs,
    'iou_types'     : iou_types,
}

# ===================================================================================== #
# Convert dictionary to `Namespace` it is then easier to load from checkpoint
# ===================================================================================== #
hparams= argparse.Namespace(**hparams_dict)

**Configuration for `pl.Trainer`:**

In [ ]:
# Create configurations for the Trainer Flags :

# ===================================================================================== #
# Insantiate Logger to Log Training logs :
# ===================================================================================== #
# Wandb logger: assuming wandb is set-up [Optional]
wb_name = f"{time.strftime('%d-%m-||-%I.%M.%S%-p')}" # change the run name here
wb_p = "pascal-2007" # change the project name here
wb_logger = WandbLogger(name=wb_name, project=wb_p, anonymous="allow",)

# learning_rate logger:
lr_logger = LearningRateLogger(logging_interval="step") # we use this to log the learning rate

# setup callbacks & loggers into a list
# since pl.Trainer expects them to be in a list format
logger=[wb_logger]
callbacks=[lr_logger]

# ===================================================================================== #
# Insantiate CheckPoint Callback :
# ===================================================================================== #
# checkpoint callback will save the top 2 checkpoints
# path to the directory where to save the checkpoints
fname =f"/content/drive/My Drive/pascal_checkpoints/{time.strftime('%d-%m-||-%I.%M.%S%-p')}"
os.makedirs(fname, exist_ok=True)
checkpoint_callback = ModelCheckpoint(fname, mode="max", monitor="valid_mAP", save_top_k=1,)

# ===================================================================================== #
# Insantiate EarlyStopping Callback :
# ===================================================================================== #
early_stop_callback = EarlyStopping(mode="max", monitor="valid_mAP", patience=3,)

# ===================================================================================== #
#  Additional Trainer Flags:
# ===================================================================================== #
check_val_every_n_epoch = 10 # Validaiton Check Interval
gpus = 1  # gpus to use
precision = 16 # precision

# ===================================================================================== #
# Create Dictionary to the Trainer Flags:
# ===================================================================================== #
trainer_config_dict = {
    'num_sanity_val_steps'   : 0,
    'benchmark'              : True, # Set benchmark True to get better performance
    'weights_summary'        : None,
    'deterministic'          : True,
    'terminate_on_nan'       : True,
    'logger'                 : logger,
    'callbacks'              : callbacks,
    'checkpoint_callback'    : checkpoint_callback,
    'early_stop_callback'    : early_stop_callback,
    'gpus'                   : gpus,
    'precision'              : precision,
    'max_epochs'             : EPOCHS,
    'check_val_every_n_epoch': check_val_every_n_epoch,
}

# ===================================================================================== #
# Convert dictionary to `Namespace`
# ===================================================================================== #
trainer_config = argparse.Namespace(**trainer_config_dict)

## **Train, validate & test:**

**Load trainer & model from the configuration files:**

In [ ]:
# Load in lightning module from the scpecified hparams
retinanet = DetectionModel(model, hparams)

# Load in trainer using trainer configs
trainer = pl.Trainer.from_argparse_args(trainer_config)

**In `PyTorchLightning` training, evaluation, validation , inference ... all is done using the special `Trainer` class.**

**Fit `RetinaNet` on the train data:** 

In [ ]:
# Fit Model
trainer.fit(retinanet)

**Evaluate trained model on the test dataset:**

In [ ]:
# Evaluate model on test dataloader
trainer.test()

**Save model weights:**

In [ ]:
## Set path to where the model should be saved
pth = os.path.join(fname, "model.pt")
torch.save(retinanet.model.state_dict(), pth)

### **Inference:**

**Imports & helper functions for inference::**

In [ ]:
from google.colab import files

# Transformations for the Inference Image:
infer_tfms = A.Compose([A.ToFloat(max_value=255.0, always_apply=True), ToTensorV2(always_apply=True),])

**Helper Functions:**

In [ ]:
# ===================================================================================== #
# Get Image Predictions :
# ===================================================================================== #
@torch.no_grad()
def get_preds(path:str, threshold:float=0.5, fname:str=pth, device:Union[str, torch.device]='cpu'):
    """
    Fn to Get predictions on given image

    Arguments :
    ----------
        1. path(str)        : path to the input Image
        2. threshold(float) : score threshold to filter predictions. Predicitons
                              with score less than threshold are discarded.
        3. fname(str)       : path to where model weights are saved. Weights should be a state_dict format
        4. device(str or torch.divice): device where to load the model
    """
    ## Parameters to Load in the Model
    param_dict = {'num_classes'  : NUM_CLASSES, 'backbone_kind': BACKBONE,}
    # Instantiate the model
    model = Retinanet(**param_dict)
    # Load in the pretrained model weights from weights file
    model.load_state_dict(torch.load(fname))
    # model to correct device
    if model.device != device:
        model.to(device)
    model.eval() # Set model to eval mode to get bbox predicitons
    # Load in the given immage fromm the Image Path
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    # Process the image
    img = infer_tfms(image=img)["image"]
    img = img.to(device)
    # Generate predictions
    pred = model([img])
    # Unpack predictions
    pred_boxes,pred_class,pred_score = pred[0]["boxes"],pred[0]["labels"],pred[0]["scores"]
    # Grab the predictions greater than the Given threshold
    pred_mask = pred_score > threshold # mask to filter predicitons based on score
    # convert predictions to numpy arrays -> List from Tensors
    boxes = list(pred_boxes[pred_mask].cpu().numpy())
    clas = list(pred_class[pred_mask].cpu().numpy())
    scores = list(pred_score[pred_mask].cpu().numpy())
    return boxes, clas, scores

# ===================================================================================== #
# Load Image and Draw predicted bounding box over the Image :
# ===================================================================================== #
## Fuction to load in the Image , get bbounding-box predictions
## and draw the bounding box predictions over the Image.
def object_detection_api(img_path:str=None, device:Union[str, torch.device]='cpu', sc_thrs:float=0.5):
    "Draw bbox predictions on given image at img_pth"
    # if Image path is not Given load image path from the user
    if img_path is None:
        uploaded = files.upload()
        img_path = list(uploaded.keys())[0]

    bb, cls, sc = get_preds(img_path, sc_thrs, device=device,)
    # Load in the Image and draw the predicted bboxes over it
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    viz.draw_bboxes(img, boxes=bb, classes=cls, scores=sc)

**Test model predictions:**

In [ ]:
idx = np.random.randint(0, len(tst_df)) # Grab the Id of a random image fromm the Test Dataset
# Detect Objects in the Given Image
object_detection_api(device='cuda:0', sc_thrs=0.6, img_path=tst_df["filename"][idx],)

In [ ]:
idx = np.random.randint(0, len(tst_df)) # Grab the Id of a random image fromm the Test Dataset
# Detect Objects in the Given Image
object_detection_api(device='cuda:0', sc_thrs=0.6, img_path=tst_df["filename"][idx],)

In [ ]:
idx = np.random.randint(0, len(tst_df)) # Grab the Id of a random image fromm the Test Dataset
# Detect Objects in the Given Image
object_detection_api(device='cuda:0', sc_thrs=0.6, img_path=tst_df["filename"][idx],)

In [ ]:
idx = np.random.randint(0, len(tst_df)) # Grab the Id of a random image fromm the Test Dataset
# Detect Objects in the Given Image
object_detection_api(device='cuda:0', sc_thrs=0.6, img_path=tst_df["filename"][idx],)